# *Part Of Speech* usando un Perceptrón Estructurado

Construir un perceptron estructurado y entrenarlo para hacer una predicción de *Part Of Speech* usando el dataset [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003). Es un dataset que contiene secuencias de frases en ingles extraídas de libros y la entidad de cada palabra (nombre, vervo, determinante, etc.) correspondiente al POS.


## Objetivos:

#### 1. Entrenar un perceptrón estructurado para predecir Part Of Speech usando el dataset ConLL.

Además, responder a las siguientes preguntas:

* 1.1. ¿Cuántos features tiene el feature mapper? ¿Qué representan?
* 1.2. En una secuencia de entrenamiento, ¿cuántos tipos de features encontramos en una secuencia? ¿Qué nos indican?
* 1.3. Cuando construimos el SP, ¿cuántos estados posibles tiene y por qué?
* 1.4. Cuando construimos el SP, ¿cuántos parámetros tiene y por qué?


#### 2. Comparar los resultados con el HMM entrenado con el mismo dataset usado en la sesión 2 en clase.


#### 3. Comprovar si el perceptrón estructurado clasifica correctamente una palabra que no ha visto en el entrenamiento.





**Part Of Speech (POS)**

- Part Of Speech (POS) se refiere a la categoría gramatical o función sintáctica que desempeña una palabra en una frase. 
- Es un concepto lingüístico utilizado para clasificar las palabras en función de sus propiedades sintácticas y morfológicas. 
- El etiquetado POS consiste en asignar una etiqueta específica a cada palabra de una frase.


## Libraries 

In [23]:
import os, sys, inspect
import numpy as np
import skseq
import skseq.sequences
from skseq.sequences import sequence
import skseq.readers
import skseq.readers.pos_corpus
import skseq.sequences.structured_perceptron as spc

# To import modules or packages that are located in a directory above the current script's directory
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

In [3]:
# to read and handle part-of-speech tagging datasets
corpus = skseq.readers.pos_corpus.PostagCorpus()

# path to the directory where the CoNLL-2003 dataset is stored
data_path = "conll"

# train_seq: a list of sequences where each sequence represents a sentence in the training data, 
            # and each word in the sentence is paired with its corresponding part-of-speech tag.
    # read_sequence_list_conll method: to read the training data from the CoNLL-2003 dataset
    # max_sent_len: An optional argument that specifies the maximum sentence length to consider. 
                    # In this case, it is set to 100.
    # max_nr_sent: An optional argument that specifies the maximum number of sentences to read. 
                   # Here, it is set to 5000.
train_seq = corpus.read_sequence_list_conll(data_path + "/train-02-21.conll", 
                                            max_sent_len=100, 
                                            max_nr_sent=5000)

test_seq = corpus.read_sequence_list_conll(data_path + "/test-23.conll", 
                                           max_sent_len=100, 
                                           max_nr_sent=1000)

# dev_seq: a list of sequences representing the development sentences along with their 
           # corresponding part-of-speech tags.
dev_seq = corpus.read_sequence_list_conll(data_path + "/dev-22.conll", 
                                          max_sent_len=100, 
                                          max_nr_sent=1000)

In [4]:
train_seq[0]

0/0 1/1 2/2 3/3 4/2 5/0 6/4 7/1 8/2 9/4 10/0 11/2 12/5 13/2 14/2 15/4 6/4 16/6 17/2 18/6 19/1 20/2 21/0 22/2 23/2 24/4 9/4 25/2 26/7 27/2 28/4 24/4 19/1 29/2 5/0 30/2 24/4 31/6 32/0 33/2 34/2 24/4 35/6 36/8 37/6 38/5 39/2 40/2 41/4 

- Primera secuencia de los datos de entrenamiento.
- Cada token de la secuencia se representa como 'word_index/tag_index', donde 'word_index' es el índice de la palabra en la frase y 'tag_index' es el índice de la part-of-speech tag correspondiente a esa palabra. 
- Por ejemplo, '0/0' indica que la primera palabra de la frase tiene asignada la part-of-speech tag con índice 0.


In [5]:
train_seq.y_dict

{'adp': 0,
 'det': 1,
 'noun': 2,
 'num': 3,
 '.': 4,
 'prt': 5,
 'verb': 6,
 'conj': 7,
 'adv': 8,
 'pron': 9,
 'adj': 10,
 'x': 11}

El diccionario `train_seq.y_dict` representa el diccionario que asigna part-of-speech tags a sus índices numéricos correspondientes. A cada part-of-speech tag se le asigna un índice único con fines de representación y cálculo en el modelo.

En el diccionario proporcionado tenemos lo siguiente: 

- 'adp': 0 representa la part-of-speech tag "adp" (preposición).
- 'det': 1 representa la part-of-speech tag "det" (determinante).
- 'noun': 2 representa la part-of-speech tag "noun" (sustantivo).
- 'num': 3 representa la parte de la etiqueta "num" (numeral).
- '.': 4 representa la part-of-speech tag '.' (signo de puntuación).
- 'prt': 5 representa la part-of-speech tag "prt" (partícula).
- 'verb': 6 representa la part-of-speech tag "verb" (verbo).
- 'conj': 7 representa la part-of-speech tag "conj" (conjunción).
- 'adv': 8 representa la part-of-speech tag "adv" (adverbio).
- 'pron': 9 representa la part-of-speech tag "pron" (pronombre).
- 'adj': 10 representa la part-of-speech tag "adj" (adjetivo).
- 'x': 11 representa la part-of-speech tag "x" (otro).

Este diccionario permite una correspondencia eficaz entre las etiquetas y sus representaciones numéricas durante el entrenamiento y la evaluación del modelo.

In [6]:
feature_mapper = skseq.sequences.id_feature.IDFeatures(train_seq)
feature_mapper

- La clase `IDFeatures` se encarga de asignar las secuencias de palabras de entrada y sus part-of-speech tags a un conjunto de características que utilizará el modelo de ML, como el perceptrón, para realizar predicciones. Estas características capturan información relevante sobre los datos de entrada que puede ayudar en la tarea de predicción.

- Al inicializar `IDFeatures` con los datos de `train_seq`, el feature mapper analiza las secuencias de entrenamiento y extrae de ellas las características necesarias. Examina cada palabra de las secuencias y genera una representación de características basada en varias propiedades lingüísticas, como la propia palabra, sus palabras vecinas, su posición en la frase y otra información contextual.

- El objeto `feature_mapper` sirve de puente entre las secuencias de entrada y el modelo de ML, permitiendo al modelo acceder a las características extraídas durante el entrenamiento o la predicción.

In [7]:
feature_mapper.build_features()

- `build_features()`: construir la representación de características para las secuencias de entrada.
- Al llamar a `build_features()`, el objeto `feature_mapper` procesa las secuencias de entrenamiento y prepara la representación de características, que se utilizará durante el entrenamiento y la predicción para proporcionar la información necesaria al modelo de aprendizaje automático.

- Por tanto, `feature_mapper.build_features()` desencadena la construcción de la representación de características para las secuencias de entrada, permitiendo al modelo aprender y hacer predicciones basadas en las características extraídas.

In [8]:
# Número de características únicas que han sido extraídas y almacenadas 
# en el atributo `feature_dict` del objeto `feature_mapper`.
len(feature_mapper.feature_dict)

15377

**1.1. Cuántos features tiene el feature mapper y qué representan:**
-  Durante el proceso de construcción de características, el `feature_mapper` analiza las secuencias de entrada e identifica diferentes patrones, propiedades o información contextual que pueden ser utilizados como características para el modelo de aprendizaje automático. A cada característica se le asigna un índice numérico y se almacena en el `feature_dict`.

- Evaluando `len(feature_mapper.feature_dict)`, se obtiene el recuento de características únicas que se han extraído de las secuencias de entrenamiento. En este caso 15377, lo que indica que hay 15377 características distintas que serán utilizadas por el modelo para el entrenamiento y la predicción.

- Contar con un mayor número de características permite al modelo captar detalles más precisos y mejorar potencialmente su rendimiento. Sin embargo, también puede aumentar la complejidad computacional y requerir más datos para generalizar eficazmente. Por lo tanto, lograr un equilibrio entre el número de características y los datos disponibles es crucial en las tareas de ML.

In [18]:
feature_list = list(feature_mapper.feature_dict)
print(feature_list[:10])

['init_tag:adp', 'id:In::adp', 'id:an::det', 'prev_tag:adp::det', 'id:Oct.::noun', 'prev_tag:det::noun', 'id:19::num', 'prev_tag:noun::num', 'id:review::noun', 'prev_tag:num::noun']


- `feature_list` contiene todas las características únicas extraídas de las secuencias de entrenamiento.
- 10 primeros elementos de la lista `feature_list`.
- Cada característica sigue un formato específico, que suele constar de una o varias partes separadas por dos puntos (`:`):

    - `init_tag:adp`: la secuencia comienza con una palabra etiquetada como una preposición.

    - `id:In::adp`: la palabra "In" es etiquetada como una preposición.

    - `prev_tag:adp::det`: la palabra actual es un determinante y la palabra anterior es una preposición. 

    - `id:oct.::noun`: la palabra "oct." es etiquetada como un sustantivo.

    - `prev_tag:det::noun`: la palabra actual es un sustantivo y la palabra anterior es un determinante.  

    - `id:19::num`: el número "19" está etiquetado como un número en la secuencia. 

    - `prev_tag:noun::num`: la palabra actual es un número y la palabra anterior es un sustantivo.

    - `id:review::noun`: la palabra "review" es etiquetada como un sustantivo. 

    - `prev_tag:num::noun`: la palabra actual es un sustantivo y la anterior un número. 

- Estas características están diseñadas para capturar distintos aspectos de los datos de entrada que son relevantes para la tarea de etiquetado de la part-of-speech.

In [19]:
set([x.split(":")[0] for x in feature_mapper.feature_dict.keys()])

{'final_prev_tag', 'id', 'init_tag', 'prev_tag'}

- Obtenemos un conjunto de las partes iniciales de las claves en el diccionario `feature_mapper.feature_dict`.

**1.2. En una secuencia de entrenamiento, ¿cuántos tipos de features encontramos en una secuencia? ¿Qué nos indican?**

En una secuencia de entrenamiento obtenemomos 4 tipos de features: 

- `final_prev_tag`: representa la última etiqueta de POS previa en una secuencia.

- `id`: identificador de una palabra específica.

- `init_tag`: representa la etiqueta de POS inicial de una secuencia.

- `prev_tag`: representa la etiqueta de POS previa o anterior de una palabra en la secuencia.


In [20]:
print ("Initial features:   ",     feature_mapper.feature_list[0][0])
print ("Transition features:",  feature_mapper.feature_list[0][1])
print ("Final features:     ",       feature_mapper.feature_list[0][2])
print ("Emission features:  ",    feature_mapper.feature_list[0][3])

Initial features:    [[0]]
Transition features: [[3], [5], [7], [9], [11], [13], [15], [5], [18], [20], [22], [24], [26], [28], [18], [30], [32], [34], [36], [38], [5], [11], [22], [28], [18], [30], [45], [47], [49], [18], [30], [15], [5], [11], [22], [18], [32], [55], [22], [28], [18], [32], [60], [62], [64], [26], [28], [18]]
Final features:      [[68]]
Emission features:   [[1], [2], [4], [6], [8], [10], [12], [14], [16], [17], [19], [21], [23], [25], [27], [29], [12], [31], [33], [35], [37], [39], [40], [41], [42], [43], [17], [44], [46], [48], [50], [43], [37], [51], [10], [52], [43], [53], [54], [56], [57], [43], [58], [59], [61], [63], [65], [66], [67]]


- Características que capturan la información relacionada con el estado inicial, las transiciones, el estado final y las emisiones en el modelo de perceptrón estructurado para el etiquetado de POS.

## Entrenamiento del perceptrón estructurado

In [21]:
# datos de entrada o entrenamiento
trainx = train_seq.x_dict
# etiquetas de salida de entrenamiento
trainy = train_seq.y_dict

- Diccionarios para obtener las representaciones numéricas de las secuencias de entrada (x) y las etiquetas de POS correspondientes (y).

In [24]:
# instancio perceptrón estructurado
sp = spc.StructuredPerceptron(trainx, 
                              trainy, 
                              feature_mapper) 

In [25]:
sp.get_num_states(), sp.get_num_observations()

(12, 16937)

**1.3. Cuando construimos el SP, ¿cuántos estados posibles tiene y por qué?**
-  Hay 12 estados posibles en el modelo del perceptrón estructurado, por tanto, hay 12 posibles etiquetas de POS en el modelo, como hemos visto anteriormente.
- Además, hay 16937 posibles características (observaciones) que el modelo puede tener en cuenta durante el entrenamiento y la inferencia.

In [26]:
# número total de características en el feature mapper
feature_mapper.get_num_features()

15377

In [27]:
sp.parameters

array([0., 0., 0., ..., 0., 0., 0.])

In [28]:
len(sp.parameters)

15377

**1.4. Cuando construimos el SP, ¿cuántos parámetros tiene y por qué?**
- Cuando construimos el perceptrón estructurado (SP), tiene un total de 15377 parámetros. Esto se debe a que cada característica en el feature mapper contribuye con un parámetro en el modelo.
- El número de parámetros coincide con el número de características en el feature mapper.



In [29]:
def evaluate_corpus(sequences, sequences_predictions):
    total = 0.0
    correct = 0.0
    for i, sequence in enumerate(sequences):
        pred = sequences_predictions[i]
        for j, y_hat in enumerate(pred.y):
            if sequence.y[j] == y_hat:
                correct += 1
            total += 1
    return correct / total # accuracy

In [30]:
# Decodificación Viterbi en las secuencias de train y test.
# Genera secuencias de predicciones (pred_train y pred_test) 
# utilizando el modelo de perceptrón estructurado. 
pred_train = sp.viterbi_decode_corpus(train_seq)
pred_test = sp.viterbi_decode_corpus(test_seq)

In [32]:
# Evaluamos las secuencias de train y test utilizando las secuencias de predicciones generadas. 
# Los resultados de evaluación se almacenan en las variables eval_train y eval_test, respectivamente.
eval_train = evaluate_corpus(train_seq.seq_list, pred_train)
eval_test = evaluate_corpus(test_seq.seq_list, pred_test)
print("Train accuracy: %.3f \tTest accuracy: %.3f"%(eval_train, eval_test))

Train acc: 0.103 	Test acc: 0.105


- Ambas accuracy son bajas, lo que indica que el modelo actual tiene un rendimiento deficiente en la tarea de etiquetado de POS en estos conjuntos de datos.

Entrenamos el modelo durante 4 epochs: 

In [33]:
%%time
num_epochs = 4
sp.fit(feature_mapper.dataset, num_epochs)

Epoch: 0 Accuracy: 0.822854
Epoch: 1 Accuracy: 0.904985
Epoch: 2 Accuracy: 0.925024
Epoch: 3 Accuracy: 0.937884
CPU times: user 50.3 s, sys: 229 ms, total: 50.6 s
Wall time: 50.6 s


- La accuracy aumenta a medida que avanzan las epochs, lo que indica que el modelo está mejorando su rendimiento en la tarea de etiquetado de POS. La accuracy inicial es del 82% en la primera epoch y alcanza alrededor del 94% en la cuarta epoch.

In [34]:
# Secuencias de etiquetas predichas por el modelo 
# para cada secuencia de entrada en los conjuntos de train y test.
pred_train = sp.viterbi_decode_corpus(train_seq)
pred_test = sp.viterbi_decode_corpus(test_seq)

- Después de entrenar el modelo de perceptrón estructurado, se utilizan las funciones `viterbi_decode_corpus()` para realizar la decodificación Viterbi en las secuencias de entrenamiento (train_seq) y prueba (test_seq). Estas funciones aplican el algoritmo de Viterbi para encontrar la secuencia de etiquetas más probable para cada secuencia de entrada, utilizando el modelo entrenado.

In [35]:
# Calculamos la accuracy del modelo utilizando la función evaluate_corpus()
eval_train = evaluate_corpus(train_seq.seq_list, pred_train)
eval_test = evaluate_corpus(test_seq.seq_list, pred_test)
print("Train acc: %.3f \tTest acc: %.3f"%(eval_train, eval_test))

Train acc: 0.921 	Test acc: 0.889


- La accuracy del modelo es del 92% en el conjunto de entrenamiento y 89% en el conjunto de prueba. 
- Esto indica que el modelo tiene un buen rendimiento en la tarea de etiquetado de POS.

**2. Comparar los resultados con el HMM entrenado con el mismo dataset usado en la sesión 2 en clase.**

Los resultados del modelo de perceptrón estructurado son diferentes de los resultados del modelo HMM entrenado con el mismo dataset.

Para el modelo de perceptrón estructurado, la accuracy en el conjunto de entrenamiento es del 92% y en el conjunto de prueba es del 89%.

En cambio, para el modelo HMM, la accuracy en el conjunto de entrenamiento y prueba es del 97%.

Esto indica que el modelo HMM obtuvo mejores resultados en términos de accuracy en el mismo dataset: tuvo una mayor proporción de predicciones correctas en relación con el total de predicciones en el dataset utilizado.


## Predecir palabras no existentes en el corpus (dataset de entrenamiento).

**3. Comprovar si el perceptrón estructurado clasifica correctamente una palabra que no ha visto en el entrenamiento.**

In [51]:
new_seq = ['walk', 'shop', 'cacophony', 'bamboozle']
new_seq_ids = [train_seq.x_dict[w] for w in new_seq]

KeyError: 'cacophony'

In [52]:
new_seq = ['walk', 'shop', 'bamboozle', 'cacophony']
new_seq_ids  = [train_seq.x_dict[w] for w in new_seq]

KeyError: 'bamboozle'

- Los errores se deben a que las palabras 'cacophony' y 'bamboozle' no están presentes en el conjunto de datos de entrenamiento. Esto significa que el perceptrón estructurado no ha visto estas palabras durante el entrenamiento y, por lo tanto, no tiene una representación numérica asignada para ellas.

In [48]:
new_seq = skseq.sequences.sequence.Sequence(x=new_seq, 
                                            y=[int(0) for w in new_seq])
new_seq

walk/0 shop/0 bamboozle/0 cacophony/0 

- Como las palabra `bamboozle` y `cacophony` no se encuentran en el diccionario construido a partir de las secuencias de entrenamiento, no podemos codificarlos usando el mismo método, pero si podemos trabajar directamente con las palabras y asignar un estado aleatorio `0`. De esta forma, podemos también obtener los features asociados a la secuencia y por lo tanto hacer una predicción del estado asociado también a las nuevas palabras:

In [49]:
# características de la secuencia
feature_mapper.get_sequence_features(new_seq)

([[0]], [[92], [92], [92]], [[2995]], [[], [], [], []])

In [50]:
# decodificación Viterbi para obtener las predicciones de etiquetas para las palabras desconocidas
sp.viterbi_decode(new_seq)[0].to_words(test_seq, only_tag_translation=True)

'walk/verb shop/noun bamboozle/noun cacophony/noun '

- Observamos que el modelo ha predicho correctamente las palabras que estaban en el conjunto de entrenamiento (walk y shop). 
- Para las nuevas palabras (palabras desconocidas pora el modelo), el modelo ha clasificado bien cacophony como sustantivo, pero ha fallado en el verbo bamboozle, clasificándolo también como sustantivo. 